# Translating JSON input
author: Raju Penmatcha

## This is a three step process
1. Convert your JSON document into XML
2. Translate the XML document using Amazon Translate. Amazon Translate will ignore XML tags 
and will translate only content within tags
3. Convert translated XML document back to original JSON format

In [1]:
# Install json2xml
!pip install json2xml

In [2]:
# An example json document
exampleJson = '{ \
  "firstName": "John", \
  "lastName": "Smith", \
  "isAlive": true, \
  "age": 27, \
  "address": { \
    "streetAddress": "21st Street", \
    "city": "New York", \
    "state": "NY", \
    "postalCode": "10021-3100" \
  }, \
  "phoneNumbers": [ \
    { \
      "type": "home", \
      "number": "212 555-1234" \
    }, \
    { \
      "type": "office", \
      "number": "646 555-4567" \
    } \
  ], \
  "children": [], \
  "spouse": null \
}'

In [3]:
import json 
def printJson(json_data):
    json_object = json.loads(json_data)
    print(json.dumps(json_object, indent=2))

printJson(exampleJson)

{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 27,
  "address": {
    "streetAddress": "21st Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    }
  ],
  "children": [],
  "spouse": null
}


In [4]:
# Step 1: Convert json document to XML
from json2xml import json2xml
from json2xml.utils import readfromurl, readfromstring, readfromjson

data = readfromstring(exampleJson)
outputXML = json2xml.Json2xml(data, attr_type=False).to_xml()
print(outputXML)

<?xml version="1.0" ?>
<all>
	<firstName>John</firstName>
	<lastName>Smith</lastName>
	<isAlive>True</isAlive>
	<age>27</age>
	<address>
		<streetAddress>21st Street</streetAddress>
		<city>New York</city>
		<state>NY</state>
		<postalCode>10021-3100</postalCode>
	</address>
	<phoneNumbers>
		<item>
			<type>home</type>
			<number>212 555-1234</number>
		</item>
		<item>
			<type>office</type>
			<number>646 555-4567</number>
		</item>
	</phoneNumbers>
	<children/>
	<spouse/>
</all>



In [5]:
# Translation helper method
import boto3
def getTranslation(text, sourceCode, destCode):
    translate = boto3.client(service_name='translate')
    result = translate.translate_text(Text=text, 
            SourceLanguageCode=sourceCode, TargetLanguageCode=destCode)
    return result.get('TranslatedText')

In [6]:
# Step 2: Do the translation
translatedXML = getTranslation(outputXML, "en", "es")
translatedXML = translatedXML.replace("<? xml version='1.0'? >", "<?xml version='1.0' ?>")

In [7]:
translatedXML = '<?xml version="1.0"?>\n<all>\n <firstName>John</firstName>\n <lastName>Smith</lastName>\n <isAlive>Cierto</isAlive>\n <age>27</age>\n <address>\n <streetAddress>Calle 21</streetAddress>\n <city>Nueva York</city>\n <state>NY</state>\n <postalCode>10021-3100</postalCode>\n </address>\n <phoneNumbers>\n <item>\n <type>casa</type>\n <number>212 555-1234</number>\n </item>\n <item>\n <type>oficina</type>\n <number>646 555-4567</number>\n </item>\n </phoneNumbers>\n <children/>\n <spouse/>\n</all>\n'

In [8]:
# Step 3: Convert translated XML back to JSON
import xmltodict
data_dict = xmltodict.parse(translatedXML)
json_data = json.dumps(data_dict) 

In [9]:
printJson(json_data)

{
  "all": {
    "firstName": "John",
    "lastName": "Smith",
    "isAlive": "Cierto",
    "age": "27",
    "address": {
      "streetAddress": "Calle 21",
      "city": "Nueva York",
      "state": "NY",
      "postalCode": "10021-3100"
    },
    "phoneNumbers": {
      "item": [
        {
          "type": "casa",
          "number": "212 555-1234"
        },
        {
          "type": "oficina",
          "number": "646 555-4567"
        }
      ]
    },
    "children": null,
    "spouse": null
  }
}
